In [255]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
from collections import Counter, defaultdict

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [295]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.timeout = timeout
        self.guessed_letters = []
        self.n_grams =[]
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.letters=sorted(set("".join(self.full_dictionary)))
        self.current_dictionary = []
        self.probability =[0]*len(self.letters)
        self.train_n_grams(self.full_dictionary)
        

    def guess(self, word): # word input example: "_ p p _ e "
        
        #Used stackoverflow, some blogs and other online resources to understand the problem and come up with a solution.
        #Used github copilot to generate the comments and some parts of the code.
    
        #Thought process
        #Approach 1: Tried to tweak the dictionary in such a way that words containing wrongly guessed letters will be eliminated. Imporved the performance marginally
        #Approach 2: Tried using a trigram model, accuracy imrpoved to 30%
        #Approach 3: Expanded the model to a five gram model, accuracy improved to 50% (Sometimes lower)
        #Approach 4: Expanded the model to a six gram model, accuracy improved to 60%. (Major reason to include the six gram model was the guess words contained a lot ofcharacters.
        # This would not have made sense if we had smaller length words in our dataset. For this use case, it performed well.)
        
        clean_word = word[::2]
        return self.n_prob(clean_word,7)

    def max_prob_unguessed_letter(self):
        max_count = 0
        max_prob_letter = ""
        for l in self.letters:
            if l in self.guessed_letters:
                continue
            if self.n_grams.get(l, 0) <= max_count:
                continue
            max_count = self.n_grams[l]
            max_prob_letter = l
        return max_prob_letter

    def train_n_grams(self,dictionary):
        
        self.n_grams = defaultdict(int)
        k=7
        for word in dictionary:
            # store 1 - k gram for each word
            for i in range(1, k+1):
                for j in range(len(word) - i + 1):
                    if i==1 or i==2:
                        #append length to the unigram and bigram
                        key=(str)(len(word))+word[j:j+i]
                        self.n_grams[key] +=1
                        
                    self.n_grams[word[j:j+i]] +=1 
        return self.n_grams

    def n_prob(self,word,n):
        #weights for each gram
        weights=[0.05,0.05,0.1,0.1,0.2,0.2,0.3]
        probs=[0]*len(self.letters)
        letter_list=[]
        n_letter_words=[]
        while n>=0:
            count=[0]*len(self.letters)
            each_count=0
            for i in range(len(word)-n+1):
                sub_word=word[i:i+n+1]
                #if there is more than one blank space, continue
                if sub_word.count("_") != 1:
                    continue
                #if the blank space is at the end, continue
                j = sub_word.find("_")
                if j == -1:
                    continue
                for key in self.n_grams:
                    #extracting length based unigram and bigram and everything for the rest
                    if len(re.sub(r'[^a-zA-Z]','', key))==(n):
                        
                        if n==0 or n==1:
                            if key[:-(n)]!='' and (int)(key[:-(n)])==len(word):
                                n_letter_words.append(re.sub(r'[^a-zA-Z]','', key))
                            else:
                                continue
                        else:
                            x=re.sub(r'[^a-zA-Z]','', key)
                            if x!='':
                                n_letter_words.append(x)
                matching_words=[]
                #pattern to match the word
                pat = r'%s[a-z]%s' % (sub_word[0:j], sub_word[j+1:])
                matching_words = [word for word in n_letter_words if re.search(pat, word)]
                n_letter_words=[]
                letters=[word[j] for word in matching_words]
                if(letters!=[]):
                    letter_list+=(letters)
            n=n-1
            letters_l=[letter for letter in letter_list if letter not in self.guessed_letters]
            # if all guessed letters are the only filtered list, return most probable letter
            if len(letters_l) == 0:
                return self.max_prob_unguessed_letter()
            for i in letters_l:
                count[self.letters.index(i)]+=1
                each_count+=1
            
            curr_prob =[x/each_count for x in count]
            #weighing larger grams more than the smaller grams
            pr=[x*weights[n] for x in curr_prob]
            probs=[x+y for x,y in zip(probs,pr)]
        maxp=max(probs)
        return self.letters[probs.index(maxp)]
    
  
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)